# Notebook 3: comparison of linear and nonlinear decoders

In [ ]:
!pip install scipy==1.7.3

!git clone https://github.com/Mike-boop/mldecoders.git

import os
os.chdir('mldecoders')

!python setup.py install

In [ ]:
# Download the predictions from the CNSP web server 

!pip install gdown
!gdown --folder https://drive.google.com/drive/folders/1vnZ1XDAlIKQH21S8uPo66ZHXaYdXOExu

predictions_dir = "Thornton_tutorial/"

# blue peter style predictions in case things go wrong:
# predictions_dir = 'cnsp_workshop_tutorial/data/single-speaker-predictions'

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, zscore

# Intro

In part 1, you trained linear decoders to predict the speech envelope from EEG recordings. In part 2, each of you trained one of the DNNs for one participant's data, and sent us the predicted speech envelope values. In this notebook, we will compare the performances of the linear models with those of the DNNs. 

# Loading the predicted speech envelopes

The outputs of the linear models which we saved can be loaded like so (using the first participant as an example):

In [ ]:
participant = 0
ridge_filepath = os.path.join(predictions_dir, f"ridge_predictions_P{participant:02d}.npy")
ridge_predictions = np.load(ridge_filepath)

In [ ]:
ground_truth = np.load(os.path.join(predictions_dir, 'ground_truth.npy'))
print(pearsonr(ridge_predictions, ground_truth))

Similarly, the predictions of the DNNs can be loaded like so:

In [ ]:
fcnn_filepath = os.path.join(predictions_dir, f"fcnn_predictions_P{participant:02d}.npy")
fcnn_predictions = np.load(fcnn_filepath)
print(pearsonr(fcnn_predictions, ground_truth))

The time series can be compared visually:

In [ ]:
cnn_filepath = os.path.join(predictions_dir, f"cnn_predictions_P{participant:02d}.npy")
cnn_predictions = np.load(cnn_filepath)

fs = 125
t = np.arange(len(cnn_predictions))/fs
plt.plot(t, ground_truth, label='envelope')
plt.plot(t, zscore(cnn_predictions), label='reconstruction')
plt.legend()

plt.xlim(10, 30)
plt.xlabel('Time [s]')

plt.title(f'correlation: {pearsonr(ground_truth, fcnn_predictions)[0]}')

## Exercise: correlate the CNN predictions with the FCNN predictions. What do you notice? Also compare the DNN predictions with the predictions of the linear models.

# Population-level analysis

Let's collect the reconstruction accuracies (correlation coefficients) for each model and participant. We will also collect the null reconstruction scores.

In [ ]:
correlations = {'ridge': [], 'cnn': [], 'fcnn':[]}
null_correlations = {'ridge': [], 'cnn': [], 'fcnn':[]}

for participant in range(13):

    for model in ['ridge', 'cnn', 'fcnn']:

        filepath = os.path.join(predictions_dir, f"{model}_predictions_P{participant:02d}.npy")
        predictions = np.load(filepath)

        score = pearsonr(ground_truth, predictions)[0]
        null_score = pearsonr(ground_truth[::-1], predictions)[0]

        correlations[model].append(score)
        null_correlations[model].append(null_score)

In [ ]:
fig, axs = plt.subplots(1,2, tight_layout=True)

axs[0].boxplot(correlations.values(), positions = [1,2,3])
axs[0].set_ylabel('reconstruction score')
axs[0].set_xticks([1,2,3])
axs[0].set_xticklabels(correlations.keys())
axs[0].set_ylim(-0.1, 0.4)

axs[1].boxplot(null_correlations.values(), positions = [1,2,3])
axs[1].set_ylabel('null reconstruction score');
axs[1].set_xticks([1,2,3])
axs[1].set_xticklabels(correlations.keys())
axs[1].set_ylim(-0.1, 0.4)

Clearly, the reconstruction scores are much greater than the null reconstruction scores, so we can be confident that the speech envelope reconstruction is working better than chance. Interestingly, the null reconstruction scores seem to be overall slightly negative. __Exercise: can you think of why this might be? Is this a problem?__

Additionally, the two DNNs appear to perform very similarly. We should perform a quick test to see whether they are significantly better than the linear models (note that really we should be performing multiple comparison corrections here):

In [ ]:
# perform single-tailed paired t-tests

from scipy.stats import ttest_rel

print('CNN vs ridge: p =', ttest_rel(correlations['cnn'], correlations['ridge'], alternative='greater')[1])
print('FCNN vs ridge: p =', ttest_rel(correlations['fcnn'], correlations['ridge'], alternative='greater')[1])

# additionally, compare the DNNs with a two-tailed paired t-test

print('FCNN vs CNN: p =', ttest_rel(correlations['fcnn'], correlations['cnn'], alternative='two-sided')[1])

# Extension: two-speaker data

Having explored using DNNs to reconstruct the envelope of clean speech from EEG recordings, we could now look at more exciting listening conditions. In our paper, we considered competing-speakers conditions, speech-in-babble-noise conditions, and speech in an unfamiliar language.

We provide the outputs of the DNNs when they were applied to competing-speakers conditions. There were two conditions:

- __fM__: the participants focussed on the male speaker, whilst ignoring the female speaker
- __fW__: the participants focussed on the female speaker, whilst ignoring the male speaker

In this extension section, we invite you to explore this data.

The data may loaded like so:

In [ ]:
# first consider the fM condition

# get the speech envelope of the attended speaker
fM_attended_envelope = np.load('cnsp_workshop_tutorial/data/two-speakers-predictions/fM_attended_truth.npy')

# get the speech envelope of the unattended speaker
fM_unattended_envelope = np.load('cnsp_workshop_tutorial/data/two-speakers-predictions/fM_unattended_truth.npy')

# get the predicted speech envelope for the first participant
# explore changing the participant, or using 'ridge' or 'fcnn' in place of 'cnn'
participant = 1
fpath = f'cnsp_workshop_tutorial/data/two-speakers-predictions/fM_predictions_YH{participant:02d}_cnn.npy'
fM_predictions = np.load(fpath)

How well do the predictions correlate with the attended and unattended envelopes, for this participant?

In [ ]:
print("attended: ", pearsonr(fM_predictions, fM_attended_envelope)[0])
print("unattended: ", pearsonr(fM_predictions, fM_unattended_envelope)[0])

Notice that the reconstruction is more similar to the attended speech envelope than it is to the unattended speech envelope. This observation may be used for _auditory attention decoding_. We can determine an attention decoding accuracy, by dividing the data into (for example) 5-second windows, and comparing the correlations between the reconstruction and each of the attended and unattended speech envelopes:

In [ ]:
from pipeline.helpers import get_scores
fs = 125 
window = 5

# divide the time series into 5s windows, and compute the correlation coefficients in each window
ridge_attended_scores = get_scores(fM_attended_envelope, fM_predictions, batch_size=window*fs)
ridge_unattended_scores = get_scores(fM_unattended_envelope, fM_predictions, batch_size=window*fs)

In [ ]:
# plot the attended and unattended reconstruction scores
plt.boxplot([ridge_attended_scores, ridge_unattended_scores]);
plt.ylabel('reconstruction score')
plt.xticks([1,2], ['attended', 'unattended'])

# calculate the proportion of windows for which the attended reconstruction score is greater than the unattended reconstruction score
# this proportion is the attention decoding accuracy
num_windows = len(ridge_attended_scores)
num_correctly_decoded = np.sum(ridge_attended_scores > ridge_unattended_scores)
decoding_accuracy = (num_correctly_decoded/num_windows * 100)

plt.title("decoding accuracy: " + str(round(decoding_accuracy, 2)) + "%");

## Over to you!

Suggestions:

- is the reconstruction of the unattended speech stream above chance level?
- how does the attention decoding accuracy vary between participants?
- how does the attention decoding accuracy vary with window size?
- is the pearson correlation coefficient the best metric for assessing the reconstruction accuracy/decoding auditory attention?